<a href="https://colab.research.google.com/github/nafisa2023/Business-Report-Notebook2/blob/main/Business_Report_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#the
print('sdf')

sdf
